# This code is adjusted to handle the new .json format

In [2]:
import os
import numpy as np
import json
from pathlib import Path
from scipy.io import wavfile
from scipy.signal import spectrogram, windows, ellip, filtfilt
import matplotlib.pyplot as plt
import tkinter as tk

# === USER INPUT ===
folder_path = '/Users/mirandahulsey-vincent/Documents/allPythonCode/BYOD_class/data_inputs/USA5510_unsegmented_songs/55'  # 🔁 Update this to your target folder
json_path = '/Users/mirandahulsey-vincent/Documents/allPythonCode/BYOD_class/data_inputs/USA5510_unsegmented_songs/55_amplitude_modulation_detection_outputs/detected_song_segments.json'  # 🔁 Update to your JSON path

# === Set Output Folder Name ===
base_dir = os.path.dirname(folder_path)
base_name = os.path.basename(folder_path)
output_folder = os.path.join(base_dir, f"{base_name}_detected_song_minutes")
spectrogram_folder = os.path.join(output_folder, 'spectrograms')
os.makedirs(output_folder, exist_ok=True)
os.makedirs(spectrogram_folder, exist_ok=True)

# === GUI resolution helper ===
def get_screen_resolution():
    root = tk.Tk()
    root.withdraw()
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    root.destroy()
    return screen_width / 100, screen_height / 100  # inches

width_inches, height_inches = get_screen_resolution()

# === Load JSON Intervals ===
with open(json_path, 'r') as f:
    detected_intervals = json.load(f)

# === Extract detected segments from original .wav files ===
detected_segments = []
sample_rate_reference = None

for file_name, intervals in detected_intervals.items():
    wav_path = os.path.join(folder_path, file_name)
    if not os.path.exists(wav_path):
        print(f"Missing file: {file_name}")
        continue

    try:
        samplerate, data = wavfile.read(wav_path)
        if sample_rate_reference is None:
            sample_rate_reference = samplerate
        elif samplerate != sample_rate_reference:
            raise ValueError(f"Sample rate mismatch in {file_name}")

        if data.ndim > 1:
            data = data.mean(axis=1)

        for start_time, end_time in intervals:
            start_sample = int(start_time * samplerate)
            end_sample = int(end_time * samplerate)
            detected_segments.append(data[start_sample:end_sample])
    except Exception as e:
        print(f"Error reading {file_name}: {e}")
        continue

# === Concatenate and save as 1-minute chunks ===
target_samples = 60 * sample_rate_reference
concatenated = np.concatenate(detected_segments)

minute_segments = []
for i in range(0, len(concatenated), target_samples):
    chunk = concatenated[i:i + target_samples]
    if len(chunk) == target_samples:
        minute_segments.append(chunk)
        out_path = os.path.join(output_folder, f'detected_song_minute_{len(minute_segments)}.wav')
        wavfile.write(out_path, sample_rate_reference, chunk.astype(np.int16))
        print(f"🎵 Saved: {out_path}")

# === Generate spectrograms ===
def process_wav_file(file_path, output_folder, segment_duration=10, low_cut=500, high_cut=8000):
    try:
        base_name = Path(file_path).stem
        samplerate, data = wavfile.read(file_path)
        if data.ndim > 1:
            data = data.mean(axis=1)

        nyquist = samplerate / 2
        wp = [low_cut / nyquist, high_cut / nyquist]
        b, a = ellip(5, 0.2, 40, wp, btype='band')
        data = filtfilt(b, a, data)

        segment_len = int(segment_duration * samplerate)
        num_segments = min(6, int(np.ceil(len(data) / segment_len)))

        fig, axs = plt.subplots(num_segments, 1, figsize=(width_inches, height_inches), sharex=True, gridspec_kw={'hspace': 0.0})
        if num_segments == 1:
            axs = [axs]

        for i in range(num_segments):
            start = i * segment_len
            end = start + segment_len
            seg_data = np.zeros(segment_len, dtype=data.dtype)
            if start < len(data):
                seg_data[:min(segment_len, len(data) - start)] = data[start:end]

            f, t, Sxx = spectrogram(
                seg_data,
                fs=samplerate,
                window=windows.gaussian(2048, std=2048/8),
                nperseg=2048,
                noverlap=1929
            )

            Sxx_log = 10 * np.log10(Sxx + np.finfo(float).eps)
            Sxx_log_clipped = np.clip(Sxx_log, a_min=3, a_max=None)
            Sxx_log_norm = (Sxx_log_clipped - Sxx_log_clipped.min()) / (Sxx_log_clipped.max() - Sxx_log_clipped.min())
            Sxx_log_norm = np.power(Sxx_log_norm, 0.7)

            axs[i].imshow(Sxx_log_norm, aspect='auto', origin='lower',
                          extent=[0, segment_duration, f.min(), f.max()], cmap='binary')
            axs[i].set_ylabel('Freq [Hz]')
            if i == num_segments - 1:
                axs[i].set_xlabel('Time [sec]')
                axs[i].set_xticks(np.linspace(0, segment_duration, 5))

        fig.suptitle(f'{base_name} – Spectrogram (Filtered {low_cut}-{high_cut} Hz)', fontsize=14)
        fig.tight_layout()
        out_fig_path = os.path.join(output_folder, f"{base_name}_spectrogram.png")
        fig.savefig(out_fig_path, dpi=300)
        plt.close(fig)
        print(f"📈 Saved: {out_fig_path}")
    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

# === Run on saved 1-minute segments ===
segment_files = sorted(Path(output_folder).glob("detected_song_minute_*.wav"))
for f in segment_files:
    process_wav_file(f, spectrogram_folder)


AttributeError: 'list' object has no attribute 'items'

In [ ]:
import os
import numpy as np
import json
from scipy.io import wavfile
from pathlib import Path

# === CONFIG ===
#folder_path = '/Users/mirandahulsey-vincent/Documents/allPythonCode/BYOD_class/data_inputs/USA5510_unsegmented_songs/55'
#json_path = '/Users/mirandahulsey-vincent/Documents/allPythonCode/BYOD_class/data_inputs/USA5510_unsegmented_songs/55_detected_song_intervals.json'

# Output folder
output_folder = folder_path + '_detected_song_files_full_recordings'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# === Load Detected Intervals ===
with open(json_path, 'r') as f:
    detected_intervals = json.load(f)

# === Step 1: Sort the files for consistent order ===
file_names = sorted(detected_intervals.keys())

# === Step 2: Load full files with detected song ===
audio_queue = []
sample_rate_reference = None

for file_name in file_names:
    wav_path = os.path.join(folder_path, file_name)
    if not os.path.exists(wav_path):
        print(f"Missing file: {file_name}")
        continue

    try:
        samplerate, data = wavfile.read(wav_path)

        if sample_rate_reference is None:
            sample_rate_reference = samplerate
        elif samplerate != sample_rate_reference:
            raise ValueError(f"Sample rate mismatch in {file_name}")

        if data.ndim > 1:
            data = data.mean(axis=1)  # Convert to mono

        audio_queue.append(data)

    except Exception as e:
        print(f"Error reading {file_name}: {e}")
        continue

# === Step 3: Chunk into 1-minute segments ===
minute_samples = 60 * sample_rate_reference
leftover = np.array([], dtype=np.float32)
chunk_count = 0

for segment in audio_queue:
    # Combine leftover from previous file
    combined = np.concatenate([leftover, segment])

    # Chop into full 1-minute chunks
    while len(combined) >= minute_samples:
        chunk = combined[:minute_samples]
        output_filename = f'detected_song_minute_{chunk_count + 1}.wav'
        output_path = os.path.join(output_folder, output_filename)
        wavfile.write(output_path, sample_rate_reference, chunk.astype(np.int16))
        print(f"Saved: {output_path}")

        chunk_count += 1
        combined = combined[minute_samples:]  # keep the remainder

    leftover = combined  # update leftover for next file

# === Step 4: Save final leftover if non-empty
if len(leftover) > 0:
    padded = np.pad(leftover, (0, minute_samples - len(leftover)), mode='constant')
    chunk_count += 1
    output_filename = f'detected_song_minute_{chunk_count}.wav'
    output_path = os.path.join(output_folder, output_filename)
    wavfile.write(output_path, sample_rate_reference, padded.astype(np.int16))
    print(f"Saved final partial chunk: {output_path}")
